In [3]:
# Imports
import numpy as np
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)

# 1. Loading the Data

This dataset comes preloaded with Keras, so one simple command will get us training and testing data. There is a parameter for how many words we want to look at. We've set it at 1000, but feel free to experiment.

In [4]:
# Loading the data (it's preloaded in Keras)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=1000)

print(x_train.shape)
print(x_test.shape)

17465344/17464789 [==============================] - 32s 2us/step
(25000,)
(25000,)


# 2. Examining the Data

Notice that the data has been already pre-processed, where all the words have numbers, and the reviews come in as a vector with the words that the review contains. For example, if the word 'the' is the first one in our dictionary, and a review contains the word 'the', then there is a 1 in the corresponding vector.
The output comes as a vector of 1's and 0's, where 1 is a positive sentiment for the review, and 0 is negative.

In [5]:
print(x_train[0])
print(y_train[0])

[1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1


In [6]:
print(x_train.shape)
print(x_test.shape)

(25000,)
(25000,)


# 3. One-hot encoding the output

Here, we'll turn the input vectors into (0,1)-vectors. For example, if the pre-processed vector contains the number 14, then in the processed vector, the 14th entry will be 1.

In [9]:
tokenizer=Tokenizer(num_words=1000)
x_train=tokenizer.sequences_to_matrix(x_train,mode='binary')
x_test=tokenizer.sequences_to_matrix(x_test,mode='binary')



ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [10]:
print(x_train.shape)

(25000, 1000)


In [11]:
# One-hot encoding the output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(25000, 2)
(25000, 2)


# 4. Building the model architecture
Build a model here using sequential. Feel free to experiment with different layers and sizes! Also, experiment adding dropout to reduce overfitting.

In [12]:
import tensorflow as tf
from keras.layers import Dense, Dropout, Flatten
import keras.optimizers as opt
import keras.backend as K
import keras.callbacks

In [13]:
class LossHistory(keras.callbacks.Callback):
    """
    This class records the history of training.
    """
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [36]:
def build_model_architecture(input_sample=None, target=None, input_dim=1000, num_batch=50, classes=1, act_func_el='relu',act_func_last='sigmoid'):
    
    # loss history
    hist=LossHistory()
    
    # number of neurons in each layer
    
    l1_num_neuron=500
    l2_num_neuron=300
    l3_num_neuron=100
    l4_num_neuron=50
    l5_num_neuron=50
    output_neuron=classes
    
    # define dense layers with activation functions
    layer_1 = Dense(l1_num_neuron, activation=act_func_el,
                    kernel_regularizer='l2', input_dim=input_dim)
    layer_2 = Dense(l2_num_neuron, activation=act_func_el,
                    kernel_regularizer='l2')
    layer_3 = Dense(l3_num_neuron, activation=act_func_el,
                    kernel_regularizer='l2')
    layer_4 = Dense(l4_num_neuron, activation=act_func_el,
                    kernel_regularizer='l2')
    layer_5 = Dense(l5_num_neuron, activation=act_func_el,
                    kernel_regularizer='l2')
    layer_out = Dense(output_neuron, activation=act_func_last,
                    kernel_regularizer='l2')
    
    # define MLP
    
    mlp=Sequential()
    mlp.add(layer_1)
    mlp.add(Dropout(0.2))
    mlp.add(layer_2)
    mlp.add(layer_3)
    mlp.add(layer_4)
    mlp.add(layer_5)
    mlp.add(layer_out)
    
    # define optimizer (optional)
    opts = opt.SGD(lr=0.001, momentum=0-9, decay=0)
    mlp.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    
    def get_loss(y_true, y_pred, regularization=0.1):
        return K.mean(K.square(y_true - y_pred)) + regularization * additional_fea
    
    mlp.fit(input_sample, target, batch_size=num_batch,epochs=12, verbose=2, callbacks=[hist])
    
    return mlp,hist

# 5. Training The Model

In [37]:
nn, nn_hist = build_model_architecture(input_sample=x_train,target=y_train,input_dim=1000,num_batch=200,classes=2,act_func_el='relu',act_func_last='sigmoid')

Epoch 1/12
 - 9s - loss: 3.3330 - acc: 0.7522
Epoch 2/12
 - 6s - loss: 0.6799 - acc: 0.8422
Epoch 3/12
 - 6s - loss: 0.5647 - acc: 0.8482
Epoch 4/12
 - 6s - loss: 0.5354 - acc: 0.8482
Epoch 5/12
 - 6s - loss: 0.5199 - acc: 0.8528
Epoch 6/12
 - 6s - loss: 0.5075 - acc: 0.8553
Epoch 7/12
 - 6s - loss: 0.5025 - acc: 0.8552
Epoch 8/12
 - 6s - loss: 0.5035 - acc: 0.8551
Epoch 9/12
 - 6s - loss: 0.4924 - acc: 0.8580
Epoch 10/12
 - 6s - loss: 0.4852 - acc: 0.8583
Epoch 11/12
 - 7s - loss: 0.4838 - acc: 0.8590
Epoch 12/12
 - 7s - loss: 0.4809 - acc: 0.8600


# 6. Testing the Model

In [41]:
print("evaluating model on test-set...")
predicted_values = nn.evaluate(x_test,y_test, verbose=2)

#predicted_values = recover_label(predicted_values, diff, col_min)
#real_label = recover_label(y_test, diff, col_min)

#print(predicted_values)
#print(real_label)
#print(nn_hist.losses)

evaluating model on test-set...


In [42]:
print('Accuracy: ',predicted_values[1])

Accuracy:  0.85676
